# 一维数据结构Series对象

Pandas模块中有两种主要的数据结构：一维数据结构Series和二维数据结构DataFrame，这两种数据结构能处理各种常见类型的数据。其中，又以二维数据结构DataFrame最为常用。

在Pandas中，一维数据结构Series可以存储任意类型的数据，包括整数、浮点数、字符串、Python 对象等。

## Series对象的生成

Pandas模块与NumPy模块需要配合使用。导入相关模块：

In [1]:
import numpy as np

In [2]:
import pandas as pd

Series对象的构造方法为：

```python
pd.Series(data=None, index=None, dtype=None, name=None)
```

其中，各参数的含义为：
- data参数可以是列表、元组或者一维数组，也可以是字典，还可以是标量值；
- index参数是一个与data大小相同的数组或索引，表示Series对象的标记；
- 与NumPy数组一样，Series对象中的数据必须是同一类型的，不指定dtype参数时，Pandas会根据data中的数据进行推断。

### 使用数组生成

使用数组生成Series对象：

In [3]:
a = pd.Series([1, 2, 3, 4])

In [4]:
a

0    1
1    2
2    3
3    4
dtype: int64

左栏是该Series对象的标记即index参数需要指定的内容，右边是对应的数据。在不指定index参数的情况下，标记默认是RangeIndex(n)，其中n是data的长度。标记可以用`.index`属性查看：

In [5]:
a.index

RangeIndex(start=0, stop=4, step=1)

可以用标记来索引对应位置的值：

In [6]:
a[0]

1

Series对象的标记类似于字典，因此与数组不同的，Series不支持负数索引。

index参数可以不是整数：

In [7]:
a = pd.Series([1, 2, 3, 4], index=["a", "b", "c", "d"])

In [8]:
a

a    1
b    2
c    3
d    4
dtype: int64

In [9]:
a['b']

2

In [10]:
a.index

Index(['a', 'b', 'c', 'd'], dtype='object')

不过Series对象也可以用数字索引：

In [11]:
a[0]

1

### 使用字典生成

使用字典生成Series对象：

In [12]:
d = {"c": 3, "b": 2, "a": 1}

In [13]:
pd.Series(d)

c    3
b    2
a    1
dtype: int64

如果指定了index参数，Pandas会按照参数指定的顺序，从字典中依次读取相应的值，并让不存在的键对应`np.nan`：

In [14]:
a = pd.Series(d, index=['c', 'd', 'b', 'e'])

In [15]:
a

c    3.0
d    NaN
b    2.0
e    NaN
dtype: float64

### 使用标量生成

Series对象还可以通过标量生成，通过指定index参数，产生一个指定大小且值全为该标量的Series对象：

In [16]:
pd.Series(5, index=range(3))

0    5
1    5
2    5
dtype: int64

In [17]:
pd.Series(5, index=["a", "b", "c", "d"])

a    5
b    5
c    5
d    5
dtype: int64

## Series对象的使用

Series对象可以像数组或字典一样使用。

### 像数组一样使用

Series对象可以从数组中生成，也支持一些数组的操作：

In [18]:
s = pd.Series(np.random.randn(5),index=['a', 'b', 'c', 'd', 'e'])

In [19]:
s

a    0.511909
b    0.936458
c    1.273640
d    0.406360
e    0.070895
dtype: float64

虽然标记不是数字，仍然可以像数组一样按照位置顺序对它进行索引：

In [20]:
s[0]

0.5119086238174924

In [21]:
s[:3]

a    0.511909
b    0.936458
c    1.273640
dtype: float64

也可以使用布尔值进行索引：

In [22]:
s[s > s.median()]

b    0.936458
c    1.273640
dtype: float64

Series对象还支持与NumPy数组类似的高级索引，同时索引多个元素：

In [23]:
s[[4, 3, 1]]

e    0.070895
d    0.406360
b    0.936458
dtype: float64

一些NumPy函数可以直接作用在Series对象上，返回的结果还是Series对象：

In [24]:
np.exp(s)

a    1.668473
b    2.550930
c    3.573839
d    1.501342
e    1.073468
dtype: float64

### 像字典一样使用

Series对象也可以像字典一样的使用，标记就相当于字典的键，可以进行值的查询：

In [25]:
s['a']

0.5119086238174924

In [26]:
s['e'] = 12

In [27]:
s

a     0.511909
b     0.936458
c     1.273640
d     0.406360
e    12.000000
dtype: float64

可以用关键字in查看Series中是否存在某个标记：

In [28]:
'e' in s

True

In [29]:
0 in s

False

Series对象也支持用.get()方法索引不存在的标记：

In [30]:
s.get('f')

In [31]:
s.get('f', np.nan)

nan

### 数学运算和标记对齐

基础的数学运算：

In [32]:
s + s

a     1.023817
b     1.872916
c     2.547281
d     0.812719
e    24.000000
dtype: float64

In [33]:
s * 2

a     1.023817
b     1.872916
c     2.547281
d     0.812719
e    24.000000
dtype: float64

In [34]:
np.exp(s)

a         1.668473
b         2.550930
c         3.573839
d         1.501342
e    162754.791419
dtype: float64

不过数组与Series对象有一个本质上的区别。数组只有顺序没有标记，而Series对象是有标记的，两个Series对象相加时，会根据标记的值进行对齐操作。

例如，`s[1:]`的标记为b到e，而`s[:-1]`的标记为a到d，它们相加时，会首先对两个Series中各自独有的部分补上np.nan，然后再相加，从而得到：

In [35]:
s[1:]

b     0.936458
c     1.273640
d     0.406360
e    12.000000
dtype: float64

In [36]:
s[:-1]

a    0.511909
b    0.936458
c    1.273640
d    0.406360
dtype: float64

In [37]:
s[1:] + s[:-1]

a         NaN
b    1.872916
c    2.547281
d    0.812719
e         NaN
dtype: float64